In [247]:
engine_path=r"stockfish\stockfish-windows-x86-64-avx2.exe"
import pandas as pd
import chess
import chess.pgn
import numpy as np
import os
import joblib
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import pickle
from datetime import datetime
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.utils import to_categorical

MAX_GAMES=15

In [248]:


def get_last_clustering_date(player_id,file_path=r"C:\Users\yassi\OneDrive\Desktop\The end\ids.txt"):

    with open(file_path, "r") as file:
        for line in file:
            if line.startswith(f"ID:{player_id},"):
                parts = line.split(", ")
                for part in parts:
                    if part.startswith("last_clustering:"):
                        date_str = part.split(":", 1)[1].strip()
                        return datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")
    return None  


In [249]:
def win_rate(player_df):
    sumWin = player_df[player_df['Result'] == 1].shape[0]
    total_games=player_df.shape[0]
    winRate=sumWin/total_games
    return winRate

def rate_per_move(player_df,var):
    sumVar=player_df[var].sum()
    sumMoves = player_df['Moves'].apply(lambda x: len(x.split())).sum() //2
    DPS=sumVar/sumMoves
    return DPS



In [250]:
def get_improvement(player_df):
    player_df = player_df.sort_values(by='Date', ascending=False)
    latest_20_games = player_df.head(20)

    df_10_most_recent = latest_20_games.iloc[:10].reset_index(drop=True)
    df_10_older = latest_20_games.iloc[10:].reset_index(drop=True)

    winRate_recent = win_rate(df_10_most_recent)
    DPS_recent = rate_per_move(df_10_most_recent, "optimal_moves")
    MistakePerMove_recent = rate_per_move(df_10_most_recent, "mistakes")
    MissPerMove_recent = rate_per_move(df_10_most_recent, "miss")
    brilliantPerMove_recent = rate_per_move(df_10_most_recent, "brilliants")
    BlunderPerMove_recent = rate_per_move(df_10_most_recent, "blunders")
    ForksMade_recent = rate_per_move(df_10_most_recent, "forkes_and_pins")

    winRate_older = win_rate(df_10_older)
    DPS_older = rate_per_move(df_10_older, "optimal_moves")
    MistakePerMove_older = rate_per_move(df_10_older, "mistakes")
    MissPerMove_older = rate_per_move(df_10_older, "miss")
    brilliantPerMove_older = rate_per_move(df_10_older, "brilliants")
    BlunderPerMove_older = rate_per_move(df_10_older, "blunders")
    ForksMade_older = rate_per_move(df_10_older, "forkes_and_pins")

    improvement = {
        "player_id":player_df["player_id"],
        "winRate": winRate_recent - winRate_older,
        "DPS": DPS_recent - DPS_older,
        "MistakePerMove": MistakePerMove_older - MistakePerMove_recent,
        "MissPerMove": MissPerMove_older - MissPerMove_recent,
        "brilliantPerMove": brilliantPerMove_recent - brilliantPerMove_older,
        "BlunderPerMove": BlunderPerMove_older - BlunderPerMove_recent,
        "ForksMade": ForksMade_recent - ForksMade_older,
    }

    return improvement

In [251]:
def determine_cluster_logic(improvement):

    win_rate_improve = improvement["winRate"]
    DPS_improve = improvement["DPS"]
    mistakes_improve = improvement["MistakePerMove"]

    # Categorize as high, balanced, or low improvement
    if win_rate_improve > 0.1 and DPS_improve > 0.1 and mistakes_improve < 0:
        return "High Improvement"
    elif -0.1 < win_rate_improve < 0.1 and -0.1 < DPS_improve < 0.1:
        return "Balanced Improvement"
    else:
        return "Low/Negative Improvement"

In [252]:
# Function to cluster players and save the clustering model
def cluster_players(cluster_data):

    scaler = StandardScaler()
    features_to_scale = ["winRate", "DPS", "MistakePerMove", "MissPerMove", "brilliantPerMove", "ForksMade"]
    scaled_features = scaler.fit_transform(cluster_data[features_to_scale])


    kmeans = KMeans(n_clusters=3, random_state=42)
    cluster_data["cluster"] = kmeans.fit_predict(scaled_features)


    cluster_labels = {
        0: "High Improvement",
        1: "Balanced Improvement",
        2: "Low/Negative Improvement", #we use feature engineering on the futur
    }
    cluster_data["cluster_name"] = cluster_data["cluster"].map(cluster_labels)


    joblib.dump(kmeans, 'models based on previous games\\kmeans_cluster_model.pkl')
    joblib.dump(scaler, 'models based on previous games\\scaler_model.pkl')

    return cluster_data


In [253]:
from datetime import datetime
import re


def update_clusters_in_file(cluster_data):

    # Get the current date
    current_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")


    with open("ids.txt", "r") as file:
        lines = file.readlines()

    cluster_mapping = dict(zip(cluster_data["player_id"], cluster_data["cluster_name"]))

    # Modify only the 'cluster' and 'last_clustering' fields for the corresponding player
    with open("ids.txt", "w") as file:
        for line in lines:
            for player_id, cluster_name in cluster_mapping.items():
                if f"ID:{player_id}" in line:
                    # Replace cluster value dynamically
                    line = re.sub(r"cluster:\S+", f"cluster:{cluster_name}", line)
                    # Update the last_clustering field
                    line = re.sub(r"last_clustering:\S+", f"last_clustering:{current_date}", line)
            file.write(line)


In [254]:



def update_player_cluster(player_id,cluster_name):

    # Get the current date
    current_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")


    with open("ids.txt", "r") as file:
        lines = file.readlines()



    # Modify only the 'cluster' and 'last_clustering' fields for the corresponding player
    with open("ids.txt", "w") as file:
        for line in lines:
                if f"ID:{player_id}" in line:
                    # Replace cluster value dynamically
                    line = re.sub(r"cluster:\S+", f"cluster:{cluster_name}", line)
                    # Update the last_clustering field
                    line = re.sub(r"last_clustering:\S+", f"last_clustering:{current_date}", line)
                file.write(line)


In [255]:


def predict_player_cluster(player_improvement, kmeans_model_path="models based on previous games\\kmeans_cluster_model.pkl", scaler_model_path="models based on previous games\\scaler_model.pkl"):

    kmeans = joblib.load(kmeans_model_path)
    scaler = joblib.load(scaler_model_path)


    player_improvement = pd.DataFrame([player_improvement])


    scaled_data = scaler.transform(player_improvement)

    predicted_cluster = kmeans.predict(scaled_data)

    return predicted_cluster[0]


In [256]:

def games_played_since(player_df, start_date):

    if isinstance(start_date, str):
        start_date = pd.to_datetime(start_date)

    # Filter the games between the given date and now
    filtered_games = player_df[player_df["Date"] >= start_date]

    # Return the number of games
    return filtered_games.shape[0]


In [257]:

def train_lstm(data,id):
    tokenizer = Tokenizer(filters='')
    tokenizer.fit_on_texts(data)
    sequences = tokenizer.texts_to_sequences(data)
    word_index = tokenizer.word_index
    vocab_size = len(word_index) + 1

    # Prepare input-output pairs
    sequence_length = 30 # Define how many moves to consider as input---------------------------------------------------
    X = []
    y = []
    for seq in sequences:
        for i in range(sequence_length, len(seq)):
            X.append(seq[i-sequence_length:i])
            y.append(seq[i])

    X = np.array(X)
    y = np.array(y)

    # One-hot encode output
    y = to_categorical(y, num_classes=vocab_size)

    # Build the LSTM model
    model = Sequential([
        Embedding(vocab_size, 64, input_length=sequence_length),
        LSTM(128, return_sequences=False),
        Dense(vocab_size, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(X, y, epochs=50, batch_size=32, validation_split=0.2)

    # Save the model
    pathLSTM=f"players_models\\playerLSTMs{id}.h5"
    model.save(pathLSTM)

    pathpkl=f"players_models\\playertokenizer{id}.pkl"
    with open(pathpkl, "wb") as file:
        pickle.dump(tokenizer, file)

In [258]:
def load_chess_games(pgn_file_path):
    games_data = []
    with open(pgn_file_path) as pgn:
        while True:
            game = chess.pgn.read_game(pgn)
            if game is None:
                break
            
            # Extract game information
            game_info = {
                'player_id': game.headers.get("id"),
                'White': game.headers.get("White"),
                'Black': game.headers.get("Black"),
                'Result': game.headers.get("Result"),
                'Date': game.headers.get("Date"),
                'brilliants': game.headers.get("brilliants"),
                'blunders': game.headers.get("blunders"),
                'forkes_and_pins': game.headers.get("forkes_and_pins"),
                'miss': game.headers.get("miss"),
                'mistakes': game.headers.get("mistakes"),
                'optimal_moves': game.headers.get("optimal_moves"),
                'Moves': ' '.join(str(move) for move in game.mainline_moves())
            }
            
            games_data.append(game_info)

    # Create a DataFrame from the list of game data
    return pd.DataFrame(games_data)

In [259]:
def Preprocess_data(df):
    df['brilliants'] = df['brilliants'].astype(int)
    df['blunders'] = df['blunders'].astype(int)
    df['forkes_and_pins'] = df['forkes_and_pins'].astype(int)
    df['miss'] = df['miss'].astype(int)
    df['player_id'] = df['player_id'].astype(int)
    df['mistakes'] = df['mistakes'].astype(int)
    df['optimal_moves'] = df['optimal_moves'].astype(int)
    for i in range(df.shape[0]):
            if (df.at[i,"White"]!="arm" 
                and df.at[i,"Result"]=="1-0") or (df.at[i,"Black"]!="arm" 
                                                  and df.at[i,"Result"]=="0-1") :
                df.at[i,"Result"]=1
                df.dropna(inplace=True)

            elif  (df.at[i,"White"]=="arm" 
                   and df.at[i,"Result"]=="1-0") or (df.at[i,"Black"]=="arm" 
                                                    and df.at[i,"Result"]=="0-1") :
                df.at[i,"Result"] = -1
                df.dropna(inplace=True)
            else:
                df.at[i,"Result"] = 0
                df.dropna(inplace=True)

            df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d %H:%M:%S')    
    return df        


In [260]:

def improve():
    df=load_chess_games(r"C:\Users\yassi\OneDrive\Desktop\The end\game.pgn")
    df=Preprocess_data(df)
    improvement_list=[]

    ids=df["player_id"].unique()
    print(ids)

    for i in ids:
        player_df = df[df['player_id'] == i]

        file_path =f"players_models\\playerLSTMs{i}.h5"

        if not os.path.exists(file_path) and player_df.shape[0]>=MAX_GAMES :#training model for the first time ù
            train_lstm(player_df["Moves"],i) 

    if len(ids)>10 and len( df['player_id'].value_counts()[df['player_id'].value_counts()%20 == 0].index.tolist())>0: #totla cluster
        for i in ids:
            player_df = df[df['player_id'] == i]
            player_improvement=get_improvement(player_df)
            improvement_list.append(player_improvement) 
            improvement_df = pd.DataFrame(improvement_list)
            cluster_data=cluster_players(improvement_df)  
            update_clusters_in_file (cluster_data)

    else: #individual cluster
        for i in ids:
            player_df = df[df['player_id'] == i]
            if games_played_since(player_df,get_last_clustering_date(i)) >20 and  os.path.exists("models based on previous games\\kmeans_cluster_model.pkl"):
                player_improvement=get_improvement(player_df)
                predicted_cluster=predict_player_cluster(player_improvement)
                update_player_cluster(i,predicted_cluster)

            
    return df    
            
   
  
            
    

In [261]:
improve()

[1 2]
2024-12-14 13:59:10
2024-12-14 14:14:44


player_id   White Black Result                Date  brilliants  blunders  \
0           1  yassin   arm     -1 2024-12-14 00:00:00           0         1   
1           1  yassin   arm     -1 2024-12-14 06:30:00           0         1   
2           1  yassin   arm     -1 2024-12-14 12:15:00           0         0   
3           2  meryem   arm     -1 2024-12-14 18:45:00           0         0   
4           2  meryem   arm     -1 2024-12-14 20:50:00           0         0   
..        ...     ...   ...    ...                 ...         ...       ...   
67          1  yassin   arm     -1 2024-12-14 06:30:00           0         1   
68          1  yassin   arm     -1 2024-12-14 12:15:00           0         0   
69          2  meryem   arm     -1 2024-12-14 18:45:00           0         0   
70          2  meryem   arm     -1 2024-12-14 20:50:00           0         0   
71          1  yassin   arm     -1 2024-12-14 01:20:00           0         0   

    forkes_and_pins  miss  mistakes  optimal_moves  \
0                 1     4         4              3   
1                 3     1         4             11   
2                 0     3         3              9   
3                 1     4         3              7   
4                 1     2         3             15   
..              ...   ...       ...            ...   
67                3     1         4             11   
68                0     3         3              9   
69                1     4         3              7   
70                1     2         3             15   
71                1     2         5              6   

                                                Moves  
0   c2c3 g8f6 g1f3 e7e6 d1c2 b7b6 d2d4 c8b7 b1d2 d...  
1   h2h3 d7d6 a2a3 g8f6 d2d4 c7c5 d4c5 b8c6 g1f3 d...  
2   d2d4 c7c5 d4d5 e7e6 c2c4 g7g6 b1c3 f8g7 e2e4 g...  
3   c2c4 g7g6 a2a3 c7c6 g2g3 d7d5 d2d4 d5c4 e2e4 b...  
4   g1f3 d7d5 d2d4 g8f6 c1g5 c8f5 e2e3 e7e6 f1d3 f...  
..                                                ...  
67  h2h3 d7d6 a2a3 g8f6 d2d4 c7c5 d4c5 b8c6 g1f3 d...  
68  d2d4 c7c5 d4d5 e7e6 c2c4 g7g6 b1c3 f8g7 e2e4 g...  
69  c2c4 g7g6 a2a3 c7c6 g2g3 d7d5 d2d4 d5c4 e2e4 b...  
70  g1f3 d7d5 d2d4 g8f6 c1g5 c8f5 e2e3 e7e6 f1d3 f...  
71  d2d4 d7d6 g1f3 b8d7 e2e4 g8f6 b1c3 a7a6 h2h3 c...  

[72 rows x 12 columns]

In [262]:

# def predict_move(id):
#     pathLSTM=f"players_models\\playerLSTMs{id}.h5"
#     pathtkl=f"players_models\\playertokenizer{id}.pkl"
#     model = load_model(pathLSTM)

#     # Load the tokenizer
#     with open(pathtkl, "rb") as file:
#         tokenizer = pickle.load(file)

#     # Example input moves
#     input_moves = ["e2e4", "e7e5", "g1f3"]
#     input_seq = tokenizer.texts_to_sequences([" ".join(input_moves)])[0]
#     input_seq = pad_sequences([input_seq], maxlen=5)  # Adjust sequence_length if needed

#     # Predict the next move
#     predicted = model.predict(input_seq)
#     predicted_move_index = np.argmax(predicted)
#     predicted_move = tokenizer.index_word[predicted_move_index]

#     print(f"Predicted next move: {predicted_move}")

# predict_move(1)

In [263]:
# from tensorflow.keras.utils import plot_model
# import matplotlib.pyplot as plt

# # Visualize the model architecture
# plot_model(
#     model,
#     to_file="model_visualization.png",
#     show_shapes=True,  # Show input/output tensor shapes
#     show_layer_names=True  # Show layer names
# )

# # Display the visualization
# img = plt.imread("model_visualization.png")
# plt.figure(figsize=(10, 10))
# plt.imshow(img)
# plt.axis("off")
# plt.show()
